---

# 🧭 JSON Schema Generation

### 🎯 Intent

Generate **standards-compliant JSON Schema** from Pydantic v2 models for **API docs, validation, and contracts** (e.g., FastAPI, OpenAPI).

---

### 🧩 Core Components

1. **📜 `.model_json_schema()`**

   * Produces a JSON Schema dict for a model.
   * Options:

     * `by_alias=True` → use field aliases
     * `ref_template` → customize `$ref` paths
     * `mode="validation"` / `"serialization"` → match input vs output rules
   * Includes `$defs` for shared/nested models.

2. **🧰 `TypeAdapter(...).json_schema()`**

   * Generates schema for plain types (`list[int]`, `Annotated[...]`).
   * Useful when endpoints accept raw arrays or tuples.

3. **🏷 Field Metadata → Schema**

   * Titles, descriptions, examples, and `deprecated=True` flow into schema.
   * Constraints (`min_length`, `max_length`, `ge`, `le`, `pattern`) map automatically.

4. **🧱 Enums, Literals, Unions**

   * `Enum` / `Literal[...]` → `enum` arrays.
   * Unions → `oneOf` / `anyOf`.
   * Discriminated unions → `oneOf` + discriminator.

5. **🔗 Aliases & Naming**

   * Aliases (`by_alias=True`) control property keys.
   * Ensures stable contracts across versions.

6. **🧩 Nested Models & Reuse**

   * Nested models referenced via `$ref` into `$defs`.
   * Avoids duplication, keeps schemas DRY.

7. **⚙️ Model Config for Schema**

   * `json_schema_extra` → inject or override schema details.
   * Extend with custom keys (`x-order`, `x-format`).

8. **🧪 Serialization-Aware Schema**

   * Use `mode="serialization"` for response schemas.
   * Ensures datetimes, UUIDs, Decimals are shown as strings.

9. **🌐 FastAPI / OpenAPI Fit**

   * FastAPI consumes `model_json_schema()` under the hood.
   * Adding titles, descriptions, examples improves docs automatically.

---


In [1]:
from __future__ import annotations
from typing import Annotated, Literal, Union
from enum import Enum
from decimal import Decimal

from pydantic import BaseModel, Field, ConfigDict, TypeAdapter

import json


# ---------- Helpers: constrained types ----------
Name = Annotated[str, Field(min_length=2, max_length=50)]
Price = Annotated[Decimal, Field(ge=0, multiple_of=Decimal("0.01"))]


# ---------- Enums ----------
class Role(str, Enum):
    admin = "admin"
    user = "user"


# ---------- Nested model ----------
class Address(BaseModel):
    street: Name
    city: Name
    zip_code: Annotated[str, Field(pattern=r"^\d{5}$")]  # 5 digits


# ---------- Main model with aliases & metadata ----------
class User(BaseModel):
    model_config = ConfigDict(
        title="User",
        json_schema_extra={"x-category": "identity"}  # custom extension
    )

    id: int = Field(alias="userId", description="Internal numeric ID")
    name: Name = Field(title="Full name", examples=["Mukesh Yadav"])
    email: Annotated[str, Field(min_length=5, pattern=r".+@.+\..+")]
    role: Role = Role.user
    address: Address | None = None


# ---------- Discriminated union example ----------
class CardPay(BaseModel):
    kind: Literal["card"]
    last4: Annotated[str, Field(pattern=r"^\d{4}$")]
    amount: Price

class UpiPay(BaseModel):
    kind: Literal["upi"]
    vpa: Annotated[str, Field(min_length=5)]
    amount: Price

Payment = Union[CardPay, UpiPay]

class Checkout(BaseModel):
    method: Payment = Field(..., discriminator="kind")


# ---------- Generate JSON Schemas ----------
if __name__ == "__main__":
    # 1) Validation schema (input shape)
    user_validation_schema = User.model_json_schema(
        by_alias=True,                # use "userId" in properties
        mode="validation"
    )

    # 2) Serialization schema (output shape)
    user_serialization_schema = User.model_json_schema(
        by_alias=True,
        mode="serialization"
    )

    # 3) Discriminated union schema (shows oneOf + discriminator)
    checkout_schema = Checkout.model_json_schema(by_alias=True, mode="validation")

    # 4) Plain type schema via TypeAdapter (e.g., list[User])
    list_of_users_schema = TypeAdapter(list[User]).json_schema()

    # 5) Another plain type example (list of constrained Decimals)
    prices_schema = TypeAdapter(list[Price]).json_schema()

    # Pretty-print
    print("# --- User (validation) ---")
    print(json.dumps(user_validation_schema, indent=2))
    print("\n# --- User (serialization) ---")
    print(json.dumps(user_serialization_schema, indent=2))
    print("\n# --- Checkout (discriminated union) ---")
    print(json.dumps(checkout_schema, indent=2))
    print("\n# --- list[User] via TypeAdapter ---")
    print(json.dumps(list_of_users_schema, indent=2))
    print("\n# --- list[Price] via TypeAdapter ---")
    print(json.dumps(prices_schema, indent=2))


# --- User (validation) ---
{
  "$defs": {
    "Address": {
      "properties": {
        "street": {
          "maxLength": 50,
          "minLength": 2,
          "title": "Street",
          "type": "string"
        },
        "city": {
          "maxLength": 50,
          "minLength": 2,
          "title": "City",
          "type": "string"
        },
        "zip_code": {
          "pattern": "^\\d{5}$",
          "title": "Zip Code",
          "type": "string"
        }
      },
      "required": [
        "street",
        "city",
        "zip_code"
      ],
      "title": "Address",
      "type": "object"
    },
    "Role": {
      "enum": [
        "admin",
        "user"
      ],
      "title": "Role",
      "type": "string"
    }
  },
  "properties": {
    "userId": {
      "description": "Internal numeric ID",
      "title": "Userid",
      "type": "integer"
    },
    "name": {
      "examples": [
        "Mukesh Yadav"
      ],
      "maxLength": 50,
      "minLength": 2,